In [1]:
#importing Library

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [3]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=fire+boltt+smart+watch&crid=17CDWYXZ95GAE&sprefix=fire+%2Caps%2C355&ref=nb_sb_ss_ts-doa-p_1_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [4]:
amazon_df

,title,price,rating,reviews,availability
0,"Fire-Boltt Tank 1.85"" Outdoor Rugged Bluetooth...","₹1,999.00",4.1 out of 5 stars,"2,088 ratings",In stock
1,"Fire-Boltt Gladiator 1.96"" Biggest Display Sma...","₹2,499.00",4.0 out of 5 stars,"2,949 ratings",In stock
2,"Fire-Boltt Ninja Call Pro Plus 1.83"" Smart Wat...","₹1,799.00",4.3 out of 5 stars,"14,437 ratings",In stock
3,Fire-Boltt Phoenix Smart Watch with Bluetooth ...,"₹1,799.00",4.3 out of 5 stars,"40,548 ratings",In stock
4,Fire-Boltt India's No 1 Smartwatch Brand Talk ...,"₹1,799.00",4.2 out of 5 stars,"33,860 ratings",In stock
5,Fire-Boltt Ninja Call Pro Smart Watch Dual Chi...,"₹1,699.00",4.2 out of 5 stars,"15,893 ratings",In stock
6,"Fire-Boltt Phoenix Pro 1.39"" Bluetooth Calling...","₹1,799.00",4.3 out of 5 stars,"40,548 ratings",In stock
7,Fire-Boltt Ninja 3 Smartwatch Full Touch 1.69 ...,"₹1,299.00",4.2 out of 5 stars,"26,347 ratings",In stock
8,"Fire-Boltt Gladiator 1.96"" Biggest Display Sma...","₹2,499.00",4.0 out of 5 stars,"2,949 ratings",In stock
9,"Fire-Boltt Ninja Call Pro Plus 1.83"" Smart Wat...","₹1,799.00",4.3 out of 5 stars,"14,437 ratings",In stock
